In [1]:
import numpy as np
import pandas as pd
import time
from sklearn.svm import SVC,LinearSVC
from sklearn.multiclass import OneVsRestClassifier


readdata = pd.read_csv("/Users/academic/Desktop/TheGrandTour/data/wineData.txt", sep="\t", header=None);
data = np.array(readdata);
data = np.delete(data, 0, 0)

data = np.swapaxes(data,0,1)


# Need to seperate the classification dimension:
classIndex = np.shape(data)[0] - 1
classification = data[classIndex]
data = np.delete(data, classIndex, axis=0)
data = data.astype(float)

# make list of colours for each number:

# make list of colours for each number:
colour = np.chararray((len(classification)),unicode = True)
for i in range(len(classification)):
    if classification[i] == '1':
        colour[i] = 'r'
    elif classification[i] == '2':
        colour[i] = 'b'
    elif classification[i] == '3':
        colour[i] = 'g'



for i in range(0, np.shape(data)[0]):
    data[i,:] = data[i,:] - np.min(data[i,:])
    data[i,:] = (data[i,:]/np.ndarray.max(data[i,:]))*2 - 1
    

def getAlpha(d,stepSize):
    """
    NEEDS IMPLEMENTATION
    Should produce 1xd(d-1)/2 array of position in grand tour.

    """
    p = d*(d-1)/2     
    primeList = []
    count = 1
    while len(primeList) < p:
        count += 1
        primeBool = False
        for i in range(2, count - 1):
            if count % i == 0:
                primeBool = True
        if primeBool == False:
            irrational = (np.sqrt(count)%1)
            primeList.append(irrational)
            
    primeList = np.asarray(primeList)
    primeList = primeList.dot(stepSize)
    """
    Irrational number generation using exponentials, not being used
    p = int(d*(d-1)/2)
    alpha = np.zeros(p) #alpha(t) parameters defining grand tour in G2,d

    for i in range(0,p):
        alpha[i] = (np.exp(i) % 1) * 2 * np.pi
        
    alpha = alpha.dot(0.001)
    """
    
    return primeList


def getAngles(alpha,d):
    """""
    Inputs: 
    alpha = 1xd(d-1)/2 array defining position on grand tour
    d = dimensions of data
    Outputs a dxd array of angles required for the transformation
    """
    theta = np.zeros((d,d));
    i = 0;
    k = 0;
    
    while i < d-1:
        j = i + 1;
        
        while j < d:
            theta[i][j] = alpha[k];
            j += 1;
            k += 1;
    
        i+= 1;
        
    return theta;


def RotationMatrix(i, j, d, theta):
    """
    Inputs:
    i = first indicie of rotating plane
    j = second indicie of rotating plane
    d = dimensions of data
    theta = dxd array of angle of rotation of rotating plane

    Outputs a rotating matrix to rotate plane of ixj plane by theta_ij
    """
    R = np.identity(d)
    R[i,i] = np.cos(theta)
    R[i,j] = -1*np.sin(theta)
    R[j,i] = np.sin(theta)
    R[j,j] = np.cos(theta)
    return R


def BetaFn(d, theta):
    """
    Inputs:
    d = dimensions of data
    theta = dxd array of angle of rotation ixj plane

    Outputs the full matrix transformation for all rotations
    """
    b = RotationMatrix(1, 2, d, theta[1,2])
    i = 1
    j = 2
    for i in range(d):
        for j in range(d):
            if j <= i:
                continue
            if i==1 and j==2:
                continue
            b = np.matmul(b, RotationMatrix(i, j, d, theta[i,j]))
            
    return b


def Transform(data,alpha):
    d = np.shape(data)[0]
    theta = getAngles(alpha,d)
    b = BetaFn(d,theta)
    a = np.matmul(b,data)
    return a
    
def SVM(a,xAxisIndex,yAxisIndex):
    
    xData = a[xAxisIndex]
    yData = a[yAxisIndex]
    nData = np.vstack((xData, yData)).T
    clf = OneVsRestClassifier(SVC(kernel='linear'), n_jobs=-1)
    clf.fit(nData, classification)
    score = clf.score(nData, classification)
    
    return score


def singleAlphaAnalysis(jlist,data,nSteps):
    """
    Alters a single parameter in the alpha vector 
    does a 2pi rotation while calculating svm at each point
    """
    xAxisIndex = 0
    yAxisIndex = 1
    d = np.shape(data)[0] #dimensions of data
    nPoints = np.shape(data)[1] #number of data points
    A = np.zeros((nSteps,d,nPoints))
    alphaMax = getAlpha(d, 1)
    dAlpha = alphaMax.dot(0)
    p = int(d*(d-1)/2)
    AL = np.zeros((nSteps,p))
    tData = np.zeros((0,d,nPoints))
    allAlpha = np.zeros((0,p))
    shortAlphaMax = np.zeros((0,p))
    score = []
    scoreMax = []
    shortScore = []
    shortTData = np.zeros((0,d,nPoints))
    alpha = alphaMax
    alphatemp = alpha
                          
    for k in range(0,p):
        j = jlist[k]
        print(j)
        alpha = alphaMax
        a = Transform(data, alpha)
        wMax = SVM(a,xAxisIndex,yAxisIndex)
        dAlpha = alpha.dot(0)
        dAlpha[j] = np.pi * 2 /(nSteps-1)
        
        for i in range(0,nSteps):
            alpha = alphaMax + i*dAlpha
            a = Transform(data,alpha)
            w = SVM(a,xAxisIndex,yAxisIndex)
            A[i] = a
            AL[i] = alpha
            score.append(w)
            if(w > wMax):
                wMax = w
                scoreMax.append(wMax)
                alphaMax = alpha
            
        tData = np.append(tData,A,axis = 0)
        
    
    return score, tData, allAlpha, shortTData, shortScore, alphaMax, scoreMax

In [2]:
nSteps = 1
iter = 2
d = np.shape(data)[0]
p = int(d*(d-1)*0.5)
totsteps = int(p*nSteps)
nPoints = np.shape(data)[1]
allScore = np.zeros((iter,totsteps))
alltData = np.zeros((iter,totsteps,d,nPoints))
allalpha = np.zeros((iter,p))
for n in range(0,2):
    jlist = jList[n]
    print(jlist)
    score, tData, allAlpha, shortTData, shortScore, alphaMax, scoreMax = singleAlphaAnalysis(jlist,data,nSteps)
    allScore[n,:] = score
    alltData[n,:,:,:] = tData
    allalpha[n,:] = alphaMax

NameError: name 'jList' is not defined

In [12]:
import sys
import numpy as np
import pyqtgraph as pg
from pyqtgraph.Qt import QtGui, QtCore


if __name__ == '__main__':
    xAxisIndex = 0
    yAxisIndex = 1
    
    app = 0
    app = QtGui.QApplication([])
    win = pg.GraphicsWindow(title = "GrandTour with SVM")
    win.setGeometry(0,0,1400,1200)
    win.setWindowTitle('GrandTour with SVM')
    pg.setConfigOptions(antialias=True) #dunno what that means

    def clickpause(pause):
        if pause == 0:
            pause = 1
        else: pause = 0
            
    btn = QtGui.QPushButton("Play/Pause",win)
    btn.clicked.connect(clickpause)
    btn.resize(500,100)
    
    w1 = win.addPlot(title="Transformed Points")
    nPoints = np.shape(tData)[2]
    s1 = pg.ScatterPlotItem(size=10, pen=pg.mkPen(None), brush=pg.mkBrush(0, 227, 17,120))
    j = 0
    pos = np.zeros((nPoints,2))
    pos[:,0] = tData[0,xAxisIndex,:]
    pos[:,1] = tData[0,yAxisIndex,:]
    s1 = pg.ScatterPlotItem(pos=pos, brush = colour, size=0.03, pxMode=False,pen=pg.mkPen(None))
    w1.addItem(s1)
    w1.setLabel(text = "Transformed Basis: " + str(xAxisIndex), axis = 'bottom' )
    w1.setLabel(text = "Transformed Basis: " + str(yAxisIndex), axis = 'left' )
    w2 = win.addPlot(title="SVM Plot") 
    w2.setLabel(text = "Tour Timestep",axis = 'bottom')
    w2.setLabel(text = "SVM Accuracy", axis = 'left')
    w2.setYRange(0,1,padding = None)
    curve = w2.plot(pen='r')
    paused = 0

    def update():
        global j, paused
        
        if j > np.shape(tData)[0]-1:
            j =0
            #j = np.argmax(score)
            paused = 0
        pos[:,0] = tData[j,xAxisIndex,:]
        pos[:,1] = tData[j,yAxisIndex,:]
        s1.setData(pos = pos,brush = colour)

        
        curve.setData(score[(j//nSteps)*nSteps:j])
        w2.setXRange(0,nSteps-1,padding = None)
        if not paused:
            j += 1
    t = QtCore.QTimer()
    t.timeout.connect(update)
    t.start(10)

## 2) Spots are transform-invariant, but not identical (top-right plot). 
## In this case, drawing is almsot as fast as 1), but there is more startup 
## overhead and memory usage since each spot generates its own pre-rendered 
## image.
    sys.exit(app.exec_())
    


/Users/karanmukhi/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


SystemExit: 0

/Users/karanmukhi/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [11]:
np.max(allScore[0])

0.9943820224719101

In [19]:
tData2 = alltData[1]

In [24]:
import sys
import numpy as np
import pyqtgraph as pg
from pyqtgraph.Qt import QtGui, QtCore


if __name__ == '__main__':
    xAxisIndex = 0
    yAxisIndex = 1
    
    app = 0
    app = QtGui.QApplication([])
    win = pg.GraphicsWindow(title = "GrandTour with SVM")
    win.setGeometry(0,0,1400,1200)
    win.setWindowTitle('GrandTour with SVM')
    pg.setConfigOptions(antialias=True) #dunno what that means

    def clickpause(pause):
        if pause == 0:
            pause = 1
        else: pause = 0
            
    btn = QtGui.QPushButton("Play/Pause",win)
    btn.clicked.connect(clickpause)
    btn.resize(500,100)
    
    w1 = win.addPlot(title="Transformed Points")
    nPoints = np.shape(tData)[2]
    s1 = pg.ScatterPlotItem(size=10, pen=pg.mkPen(None), brush=pg.mkBrush(0, 227, 17,120))
    j = 0
    pos = np.zeros((nPoints,2))
    pos[:,0] = tData[0,xAxisIndex,:]
    pos[:,1] = tData[0,yAxisIndex,:]
    s1 = pg.ScatterPlotItem(pos=pos, brush = colour, size=0.03, pxMode=False,pen=pg.mkPen(None))
    w1.addItem(s1)
    w1.setLabel(text = "Transformed Basis: " + str(xAxisIndex), axis = 'bottom' )
    w1.setLabel(text = "Transformed Basis: " + str(yAxisIndex), axis = 'left' )
    w2 = win.addPlot(title="Transformed Points")
    nPoints = np.shape(tData)[2]
    s2 = pg.ScatterPlotItem(size=10, pen=pg.mkPen(None), brush=pg.mkBrush(0, 227, 17,120))
    pos2 = np.zeros((nPoints,2))
    pos2[:,0] = tData2[0,xAxisIndex,:]
    pos2[:,1] = tData2[0,yAxisIndex,:]
    s2 = pg.ScatterPlotItem(pos=pos, brush = colour, size=0.03, pxMode=False,pen=pg.mkPen(None))
    w2.addItem(s2)
    w2.setLabel(text = "Transformed Basis: " + str(xAxisIndex), axis = 'bottom' )
    w2.setLabel(text = "Transformed Basis: " + str(yAxisIndex), axis = 'left' )
    paused = 0
    
    
    def update():
        global j, paused
        
        if j > np.shape(tData)[0]-1:
            j =0
            #j = np.argmax(score)
            paused = 0
        pos[:,0] = tData[j,xAxisIndex,:]
        pos[:,1] = tData[j,yAxisIndex,:]
        s1.setData(pos = pos,brush = colour)

        
        pos2[:,0] = tData2[j,xAxisIndex,:]
        pos2[:,1] = tData2[j,yAxisIndex,:]
        s1.setData(pos = pos,brush = colour)
        s2.setData(pos = pos,brush = colour)
        if not paused:
            j += 1
    t = QtCore.QTimer()
    t.timeout.connect(update)
    t.start(10)

## 2) Spots are transform-invariant, but not identical (top-right plot). 
## In this case, drawing is almsot as fast as 1), but there is more startup 
## overhead and memory usage since each spot generates its own pre-rendered 
## image.
    sys.exit(app.exec_())
    


SystemExit: 0

/Users/karanmukhi/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [28]:
d

13

In [29]:
alpha1 = allalpha[0] 
alpha2 = allalpha[1]

In [30]:
b1 = BetaFn(d, getAngles(alpha1,d))
b2 = BetaFn(d, getAngles(alpha2,d))

In [31]:
b1 - b2

array([[ 2.08093505e-02, -7.42866672e-02,  5.52015465e-02,
        -2.16740865e-01,  2.97198683e-02, -9.40250815e-02,
         6.06728969e-01, -4.37604157e-02, -5.61319381e-01,
         1.77328490e-01, -1.07221166e-02,  4.24765954e-01,
         5.16314404e-01],
       [ 4.02026814e-04,  1.56075781e-01,  3.33474292e-02,
         1.66883654e-01, -3.38724191e-01,  7.94278784e-02,
        -2.92930450e-01,  6.48573310e-02, -1.97734312e-01,
         2.06821192e-01, -1.36356126e-01, -6.61975408e-02,
         1.43248147e-01],
       [ 2.23786967e-02,  6.68931007e-02, -1.85973405e-01,
         1.73372299e-01,  1.75925166e-01, -2.40302368e-01,
         3.71504919e-01,  4.04931911e-01,  2.53506421e-02,
         2.02027778e-01,  2.60128355e-01,  8.39091138e-01,
        -1.04396520e-01],
       [ 7.35163279e-03,  1.33924275e-01,  1.42030229e-01,
        -4.00298403e-01,  2.04946848e-01, -6.67495758e-01,
         3.39256367e-01, -6.45501982e-02,  2.08241252e-01,
        -5.96259574e-01,  1.56240068e

In [41]:
alpha = getAlpha(4,1)
theta = getAngles(alpha,4)

In [42]:
theta

array([[0.        , 0.41421356, 0.73205081, 0.23606798],
       [0.        , 0.        , 0.64575131, 0.31662479],
       [0.        , 0.        , 0.        , 0.60555128],
       [0.        , 0.        , 0.        , 0.        ]])